In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# %cd '/content/drive/My Drive/Colab Notebooks/db'
# !ls

/content/drive/My Drive/Colab Notebooks/db
flights.db	   Meteorite_Landings.csv  test.hdf	 test.pkl
GDP_melted.csv	   poblacion.csv	   test.json	 test.xlsx
london_merged.csv  test.csv		   test.parquet


In [1]:
import pandas as pd
import numpy as np
pd.__version__

'1.2.4'

In [2]:
df_pob = pd.read_csv('../data/poblacion.csv')
df_pob

,Country,year,pop
0,Afghanistan,2015,3.441360e+07
1,Albania,2015,2.880703e+06
2,Algeria,2015,3.972802e+07
3,American Samoa,2015,5.581200e+04
4,Andorra,2015,7.801100e+04
...,...,...,...
1035,Pre-demographic dividend,2018,9.194854e+08
1036,Small states,2018,4.057532e+07
1037,South Asia,2018,1.814389e+09
1038,South Asia (IDA & IBRD),2018,1.814389e+09


In [4]:
pd.options.display.float_format = '{:,.1f}'.format

In [5]:
df_pob

,Country,year,pop
0,Afghanistan,2015,"34,413,603.0"
1,Albania,2015,"2,880,703.0"
2,Algeria,2015,"39,728,025.0"
3,American Samoa,2015,"55,812.0"
4,Andorra,2015,"78,011.0"
...,...,...,...
1035,Pre-demographic dividend,2018,"919,485,393.0"
1036,Small states,2018,"40,575,321.0"
1037,South Asia,2018,"1,814,388,744.0"
1038,South Asia (IDA & IBRD),2018,"1,814,388,744.0"


In [8]:
#convertir la columna year a variable categorica
df_pob['year'] = pd.Categorical(df_pob['year'].apply(str))

In [9]:
df_pob.dtypes

Country      object
year       category
pop         float64
dtype: object

In [10]:
#verdadero solo para los paises especificados
idx_filtro = df_pob['Country'].isin(['Aruba','Colombia'])
idx_filtro

0       False
1       False
2       False
3       False
4       False
        ...  
1035    False
1036    False
1037    False
1038    False
1039    False
Name: Country, Length: 1040, dtype: bool

In [11]:
df_sample = df_pob[idx_filtro]
df_sample

,Country,year,pop
9,Aruba,2015,"104,341.0"
42,Colombia,2015,"47,520,667.0"
269,Aruba,2016,"104,872.0"
302,Colombia,2016,"48,171,392.0"
529,Aruba,2017,"105,366.0"
562,Colombia,2017,"48,901,066.0"
789,Aruba,2018,"105,845.0"
822,Colombia,2018,"49,648,685.0"


In [13]:
#multiples indices para un mejor analisis
df_sample = df_sample.set_index(['Country','year']).sort_index()
df_sample

pop
Country  year             
Aruba    2015    104,341.0
         2016    104,872.0
         2017    105,366.0
         2018    105,845.0
Colombia 2015 47,520,667.0
         2016 48,171,392.0
         2017 48,901,066.0
         2018 49,648,685.0

In [14]:
df_sample.loc['Colombia',:]

,pop
year,
2015,"47,520,667.0"
2016,"48,171,392.0"
2017,"48,901,066.0"
2018,"49,648,685.0"


In [15]:
df_sample.loc['Colombia',:].loc['2016',:]

pop   48,171,392.0
Name: 2016, dtype: float64

In [16]:
#seleccion con xs con el indice
df_sample.xs(['Aruba'])

,pop
year,
2015,"104,341.0"
2016,"104,872.0"
2017,"105,366.0"
2018,"105,845.0"


In [17]:
#seleccion con multiples indices
df_sample.xs(['Aruba','2018'])

pop   105,845.0
Name: (Aruba, 2018), dtype: float64

In [18]:
#seleccion por indice en segundo nivel
df_sample.xs('2018', level='year')

,pop
Country,
Aruba,"105,845.0"
Colombia,"49,648,685.0"


In [19]:
#establecer multiples indices y ordenarlos
df_countries = df_pob.set_index(['Country','year']).sort_index(ascending = [True,True])
df_countries

pop
Country     year             
Afghanistan 2015 34,413,603.0
            2016 35,383,128.0
            2017 36,296,400.0
            2018 37,172,386.0
Albania     2015  2,880,703.0
...                       ...
Zambia      2018 17,351,822.0
Zimbabwe    2015 13,814,629.0
            2016 14,030,390.0
            2017 14,236,745.0
            2018 14,439,018.0

[1040 rows x 1 columns]

In [21]:
ids = pd.IndexSlice #funcion para slicing
df_countries.loc[ids['Aruba':'Austria','2015':'2017'],:].sort_index()

pop
Country   year             
Aruba     2015    104,341.0
          2016    104,872.0
          2017    105,366.0
Australia 2015 23,815,995.0
          2016 24,190,907.0
          2017 24,601,860.0
Austria   2015  8,642,699.0
          2016  8,736,668.0
          2017  8,797,566.0

In [22]:
df_countries.index.get_level_values(0) # 0 para indices de primer nivel

Index(['Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Albania',
       'Albania', 'Albania', 'Albania', 'Algeria', 'Algeria',
       ...
       'Yemen, Rep.', 'Yemen, Rep.', 'Zambia', 'Zambia', 'Zambia', 'Zambia',
       'Zimbabwe', 'Zimbabwe', 'Zimbabwe', 'Zimbabwe'],
      dtype='object', name='Country', length=1040)

In [23]:
df_countries.index.get_level_values(1) #1 para indices de segundo nivel

CategoricalIndex(['2015', '2016', '2017', '2018', '2015', '2016', '2017',
                  '2018', '2015', '2016',
                  ...
                  '2017', '2018', '2015', '2016', '2017', '2018', '2015',
                  '2016', '2017', '2018'],
                 categories=['2015', '2016', '2017', '2018'], ordered=False, name='year', dtype='category', length=1040)

In [24]:
#dataframe con multiples indices o columnas
df_countries['pop']['Colombia']['2018']

49648685.0

In [25]:
df_countries.sum(level='year')

,pop
year,
2015,"65,679,147,019.0"
2016,"66,487,930,677.0"
2017,"67,294,176,701.0"
2018,"68,087,886,692.0"


In [26]:
df_sample

pop
Country  year             
Aruba    2015    104,341.0
         2016    104,872.0
         2017    105,366.0
         2018    105,845.0
Colombia 2015 47,520,667.0
         2016 48,171,392.0
         2017 48,901,066.0
         2018 49,648,685.0

In [27]:
#convertir datos de una columnas en columnas
df_sample.unstack('year')

pop                                       
year             2015         2016         2017         2018
Country                                                     
Aruba       104,341.0    104,872.0    105,366.0    105,845.0
Colombia 47,520,667.0 48,171,392.0 48,901,066.0 49,648,685.0

In [28]:
df_sample.unstack('Country')

pop             
Country     Aruba     Colombia
year                          
2015    104,341.0 47,520,667.0
2016    104,872.0 48,171,392.0
2017    105,366.0 48,901,066.0
2018    105,845.0 49,648,685.0